In [1]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from pathlib import Path

# Global settings for matplotlib
fontsize = 8
matplotlib.rc('xtick', labelsize=fontsize) 
matplotlib.rc('ytick', labelsize=fontsize) 

plt.style.use("default")
plt.rcParams.update({
    #"text.usetex": False,
    #"font.family": "serif",
    #"font.serif": ["Helvetica"],
    'figure.dpi': 200,  # renders images larger for notebook
    'axes.grid': True,
    'grid.alpha': 0.3,

})

DATA_SOURCE = Path.cwd() / 'data' / 'input' / 'raw'
PROC_SOURCE = DATA_SOURCE.parent / "processed"

def get_bh_integral(df):
    b, h = df.loc[:, [f"B_t_{k}" for k in range(1024)]].to_numpy(), df.loc[:, [f"H_t_{k}" for k in range(1024)]].to_numpy()
    h += 100
    return df.freq * 0.5*np.abs(np.sum(b * (np.roll(h, 1, axis=1) - np.roll(h, -1, axis=1)) , axis=1))  # shoelace formula

get data in pandas df

In [2]:
ds = pd.read_pickle(PROC_SOURCE / "ten_materials.pkl.gz")
# full_b, full_h = ds.loc[:, [f"B_t_{k}" for k in range(1024)]].to_numpy(), ds.loc[:, [f"H_t_{k}" for k in range(1024)]].to_numpy()
# print(full_b.shape)

exemplary BH curve

In [3]:
opertation_point_index = 42
plt.figure(dpi=150, figsize=(2,2))
plt.plot(full_h[opertation_point_index], full_b[opertation_point_index])
plt.xlabel("H in A/m", fontsize=fontsize)
plt.ylabel("B in T", fontsize=fontsize)
plt.show()

NameError: name 'full_h' is not defined

<Figure size 300x300 with 0 Axes>

# exploring the data

In [ ]:
# Display all available materials
materials = ds.loc[:, ["material"]]["material"].unique()
print(f"Materials: {materials}")

# Display all available temperatures
temperatures = ds.loc[:, ["temp"]]["temp"].unique()
print(f"Temperatures: {temperatures}")

In [ ]:
# Choose a temperature - material pair
temperature = 25
material = 'N49'

# Display all available frequencies for the temperature - material pair
frequencies = sorted(list(ds.loc[(ds["material"] == material) & (ds["temp"] == temperature)]["freq"].unique()))
print(f"Frequencies: {frequencies}")

# filter data for the temperature - material pair
ploss = ds.loc[(ds["material"] == material) & (ds["temp"] == temperature)]["ploss"].to_numpy()
b_max = ds.loc[(ds["material"] == material) & (ds["temp"] == temperature)][[f"B_t_{k}" for k in range(0,1024, 4)]].to_numpy().max(1)
b_vecs = ds.loc[(ds["material"] == material) & (ds["temp"] == temperature)][[f"B_t_{k}" for k in range(0,1024, 4)]].to_numpy()
h_vecs = ds.loc[(ds["material"] == material) & (ds["temp"] == temperature)][[f"H_t_{k}" for k in range(0,1024, 4)]].to_numpy()
freq = ds.loc[(ds["material"] == material) & (ds["temp"] == temperature)]["freq"].to_numpy()

In [ ]:
fig, axes = plt.subplots(1, 2, dpi=200, figsize=(9, 3), sharex=True, sharey=False)


samples_per_periode = 512
div_factor = int(1024 / samples_per_periode)
b = b_vecs[:, ::div_factor]
h = h_vecs[:, ::div_factor]

# rms = np.sqrt(np.mean(b**2, axis=1)) 
# abs = np.mean(np.abs(b), axis=1)
# form_factors = rms / mean_abs
# print(rms)
# print(mean_abs)
# print(form_factors)

def form_factor(b): 
    """kf = rms / mean(abs)"""
    return np.sqrt(np.mean(b**2, axis=1))  / np.mean(np.abs(b), axis=1)

def crest_factor(b): 
    """ks = rms / max()"""
    return np.max(np.abs(b), axis=1)  / np.sqrt(np.mean(b**2, axis=1))

# filter with form factor: only keep sines
# form factor +/- interval
interval_kf = 0.005
desired_kf = np.pi/(2*np.sqrt(2))  # ideal sine

# filter with crest factor: only keep sines
# crest factor +/- interval
interval_kc = 0.005
desired_kc = np.sqrt(2)  # ideal sine

# initialize all True
filter_bool = [True] * b.shape[0]

# statements = [[crest_factor(b) < desired_kc * (1 + interval_kc)]]
statements = [list(form_factor(b) < desired_kf * (1 + interval_kf)),
              list(form_factor(b) > desired_kf * (1 - interval_kf)),
              list(crest_factor(b) < desired_kc * (1 + interval_kc)),
              list(crest_factor(b) > desired_kc * (1 - interval_kc))]

for statement in statements:
    filter_bool = [a and zr for a, zr in zip(filter_bool, statement)]
    print(np.array(filter_bool).sum())

b = b[filter_bool]
h = h[filter_bool]

b_max = b_max[filter_bool]
freq = freq[filter_bool]
ploss = ploss[filter_bool]

print(b.shape)
print(h.shape)
# print(b)
# print(h)
axes[0].plot(b.T)
axes[1].plot(h.T)

plt.show()

In [ ]:
# Plot some exemplary parts of the data
plt.figure(dpi=150, figsize=(2,2))

fig, axes = plt.subplots(1, ncols=2, dpi=150, sharex=False, sharey=True, figsize=(6, 3))

axes[0].scatter(freq, ploss)
axes[0].set_xlabel('frequency in Hz', fontsize=fontsize)
axes[0].set_ylabel("loss density in W/m³", fontsize=fontsize)
axes[1].scatter(b_max, ploss)
axes[1].set_xlabel('Bpeak in T', fontsize=fontsize)

plt.show()

# Steinmetz Fitting

In [ ]:
def steinmetz(freq, b_max, alpha, beta, k):
    return k*freq**alpha*b_max**beta

In [ ]:
# Some heuristic values as comparison
init_alpha = 1.42
init_beta = 2.6
init_k = 2

# estimated_loss = np.array(len(freq))
def estimated_loss(alpha, beta, k):
    return steinmetz(freq, b_max, alpha, beta, k)

def normalized_error(alpha, beta, k):
    return np.mean(abs((estimated_loss(alpha, beta, k)-ploss)/ploss))


normalized_error(init_alpha, init_beta, init_k)
#plt.scatter(freq, estimated_loss)
#plt.scatter(freq, ploss)
#plt.scatter(freq, (estimated_loss-ploss)/ploss)
#plt.show()

In [ ]:
import optuna

def objective(trial):
    aa = trial.suggest_float('aa', 1, 2)
    bb = trial.suggest_float('bb', 2.5, 4)
    kk = trial.suggest_float('kk', 0.1, 100)
    return normalized_error(aa, bb, kk)

study = optuna.create_study()
study.optimize(objective, n_trials=500)

study.best_params  # E.g. {'x': 2.002108042}

In [ ]:
# fig = optuna.visualization.plot_optimization_history(study)
# fig.show()


In [ ]:
# fig = optuna.visualization.plot_contour(study, params=["aa", "bb", "kk"])
# fig.show()

In [ ]:
error_norm = (estimated_loss(study.best_params["aa"],study.best_params["bb"],study.best_params["kk"]) -ploss)/ploss


# 1D error plots
#plt.scatter(b_max, error_norm)
#plt.scatter(freq, error_norm)

# 2D error plots
plt.figure(dpi=150, figsize=(3,3))
cm = matplotlib.colormaps['plasma']
xy = range(20)
sc = plt.scatter(b_max, freq, c=abs(error_norm), vmin=None, vmax=None, s=None, cmap=cm)
cbar = plt.colorbar(sc, )
cbar.set_label('normalized error: (pSE - preal)/preal', fontsize=fontsize)
plt.xlabel("Bpeak in T", fontsize=fontsize)
plt.ylabel("frequency in Hz", fontsize=fontsize)

plt.show()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
current = np.linspace(-50, 50, 200)
flux = 20e-3 * np.tanh(0.1*current)

plt.plot(current, flux)
plt.show()